# 一切皆对象

In [5]:
def hi(name='yasoob'):
    return 'hi'+ name
print(hi()) 

hiyasoob


In [6]:
# 将hi函数赋值给一个变量
greet = hi # 这里hi函数后不能添加小括号，因为我们并不是在调用hi函数
print(greet())

hiyasoob


In [7]:
# 如果我们删除旧的hi函数
del hi
# print(hi())

# NameError: name 'hi' is not defined
print(greet())

hiyasoob


# 更进一步， 在函数中定义函数

In [12]:
def hi(name='yasoob'):
    print('now you are inside the hi() function')
    
    def greet():
        return 'now you are inside the greet() function'
    def welcome():
        return 'now you are inside the welcome() functin'
    
    print(greet())
    print(welcome())
    print('now you are back in the hi() function')
    
hi()

# 该例子展示了无论何时你调用hi(),greet()和welcome()将会被同时调用
# 但greet()和welcome()函数在hi()函数之外是不能被访问的

now you are inside the hi() function
now you are inside the greet() function
now you are inside the welcome() functin
now you are back in the hi() function


# 从函数中返回函数

In [14]:
# 不需要在一个函数中执行另一个函数，也可以将其作为输出返回
def hi(name='yasoob'):
    def greet():
        return 'now you are in the greet() function'
    
    def welcome():
        return 'now you are in the welcome() function'
    if name == 'yasoob':
        return greet
    else:
        return welcome
a = hi()
print(a) # 现在a指向了hi()函数中的greet()函数
print(a())

<function hi.<locals>.greet at 0x000001E8C42372F0>
now you are in the greet() function


# 将函数作为参数传给另一个函数

In [17]:
def hi():
    return 'hi yasoob!'

# 传入一个函数作为参数，并且这个函数在doSomethingBeforeHi内部被执行
def doSomethingBeforeHi(func): 
    print('I am doing some boring work before executing hi()' )
    print(func())
    
doSomethingBeforeHi(hi)

I am doing some boring work before executing hi()
hi yasoob!


# 第一个装饰器

In [21]:
# Python中装饰器封装一个函数并修改它的行为

def a_new_decorator(a_func):
    def warpTheFunction():
        print('I am doing some boring work before executing a_func()')
        
        a_func()
        
        print('I am doing some boring work after executing a_func()')
    return warpTheFunction

def a_function_requiring_decoration():
    print('I am the function which needs some decoration to remove my foul smell')

# now a_function_requiring_decoration is wrapped by wrapTheFunction
a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration) 
a_function_requiring_decoration()

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()


# 使用@语法糖

In [23]:
# the @a_new_decorator is just a short way of saying:
# a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)

@a_new_decorator
def a_function_requiring_decoration():
    '''Hey you! Decorate me!'''
    print('I am the function which needs some decoration to remove my foul smell.')
a_function_requiring_decoration()

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()


# 关于函数名和注释文档

In [28]:
# 由于a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)
# a_function_requiring_decoration的函数名和注释文档被重写

print(a_function_requiring_decoration.__name__) # 期望的输出是a_function_requiring_decoration
print(a_function_requiring_decoration.__doc__)

warpTheFunction
None


In [32]:
# 可以使用Python的functools.wraps
# @wraps接受一个函数来进行装饰，并加入了复制函数名称、注释文档、参数列表等等的功能。
# 这可以让我们在装饰器里面访问在装饰之前的函数的属性。

from functools import wraps

def a_new_decorator(a_func):
    @wraps(a_func)
    def wrapTheFunction():
        print('I am doing some boring work before executing a_func()')
        a_func()
        print('I am doing some boring work after executing a_func()')
    return wrapTheFunction

@a_new_decorator
def a_function_requiring_decoration():
    """Hey you! Decorate me!"""
    print("I am the function which needs some decoration to "
          "remove my foul smell")
 
print(a_function_requiring_decoration.__name__)
print(a_function_requiring_decoration.__doc__)

a_function_requiring_decoration
Hey you! Decorate me!


# 装饰器的使用场景

In [33]:
# 授权(Authorization)

# 装饰器能有助于检查某个人是否被授权去使用一个web应用的端点(endpoint)
# 它们被大量使用于Flask和Django web框架中

from functools import wraps
 
def requires_auth(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        auth = request.authorization
        if not auth or not check_auth(auth.username, auth.password):
            authenticate()
        return f(*args, **kwargs)
    return decorated

In [34]:
# 日志 Logging

from functools import wraps
 
def logit(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return with_logging
 
@logit
def addition_func(x):
   """Do some math."""
   return x + x
 
 
result = addition_func(4)
# Output: addition_func was called

addition_func was called


# 带参数的装饰器

In [ ]:
from functools import wraps
 
def logit(logfile='out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile，并写入内容
            with open(logfile, 'a') as opened_file:
                # 现在将日志打到指定的logfile
                opened_file.write(log_string + '\n')
            return func(*args, **kwargs)
        return wrapped_function
    return logging_decorator
 
@logit()
def myfunc1():
    pass
 
myfunc1()
# Output: myfunc1 was called
# 现在一个叫做 out.log 的文件出现了，里面的内容就是上面的字符串
 
@logit(logfile='func2.log')
def myfunc2():
    pass
 
myfunc2()
# Output: myfunc2 was called